In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from os import listdir
from os.path import isfile, join

# some comment

embeddings = HuggingFaceEmbeddings(
    model_name = 'emilyalsentzer/Bio_ClinicalBERT'
)
 
text = "This is a test document."
query_result = embeddings.embed_query(text)


/Users/liangxin/anaconda3/envs/vector/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name emilyalsentzer/Bio_ClinicalBERT. Creating a new one with MEAN pooling.


In [2]:
#onlyfiles = [f for f in listdir('top_1000_txt') if isfile(join('top_1000_txt', f))]
onlyfiles=['note_211.txt']
raw_documents = []
for file in onlyfiles:
    print(file)
    raw_doc = TextLoader(f'top_1000_txt/{file}').load()
    raw_documents.extend(raw_doc)

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=24)

note_211.txt


In [3]:
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embeddings)

In [4]:


llm_model = "gpt-3.5-turbo"

In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [6]:

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


In [9]:

llm = ChatOpenAI(temperature=0.7, model_name=llm_model, openai_api_key="")

     

/Users/liangxin/anaconda3/envs/vector/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [13]:
retriever = db.as_retriever(search_kwargs={"k": 1})

template = """You are an expert clinical assistant. You will receive a collection of clinical notes. You will summarize them in the style of a discharge summary.Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser())

In [14]:
chain.invoke('What allergies did the patient have?')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'The patient was recorded as having No Known Allergies to Drugs.'

In [27]:
from datasets import Dataset
questions= [
    "Has the patient got a history of diabetes?",
    "What allergies did the patient have?",
    "Does the patient use tobacco or alcohol?",
    "What medication is the patient taking?",
    "What measures were taken for the patient?",
    "What imaging studies were performed on the patient?",
    "What follow-up care was scheduled for the patient?",
    "What were the results of the patient’s tests?",
    "What did C-spine show?",
    "Does the patient use alcohol?",
    "Does the patient use cigarettes or alcohol?",
    "Did patient use tobacco?"
]

ground_truths = [["No."],
                 ["The patient has no known allergies to drugs."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes) but denies alcohol use."],
                 ["Dilantin 100 IV TID."],
                 ["The patient was intubated, had a left radial arterial line placed for close BP monitoring, and was admitted to the TICU. A cervical collar was also placed and later removed after C-spine clearance."],
                 ["The patient underwent head CT, C-spine, chest/abd/pelvis CT scans, and an MRI of the C-spine"],
                 ["An outpatient CT scan was scheduled to be done in 2 weeks, and a follow-up clinic appointment was scheduled in 2 weeks after the CT scan.No need to follow up in the trauma clinic but the patient may call the clinic with any questions."],
                 ["The patient’s tests showed a diffuse axonal injury and hemorrhagic shearing injuries with post-traumatic intraventricular and cisternal hemorrhagic foci on the head CT. The CT of the C-spine, chest/abd/pelvis was all negative. The MRI of the C-spine showed no fractures or ligamentous injuries."],
                 ["The CT of the C-spine showed no fractures or ligamentous injuries. "],
                 ["No, the patient denies alcohol use."],
                 ["The patient occasionally uses cigarettes and denies alcohol use."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes)."]

]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)

In [28]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 12
})

In [29]:
dataset.save_to_disk('gpt3.5dataset')

Saving the dataset (1/1 shards): 100%|██████████| 12/12 [00:00<00:00, 1947.37 examples/s]


In [30]:
ragas_input_df = dataset.to_pandas()
display(ragas_input_df.head())

,question,answer,contexts,ground_truths
0,Has the patient got a history of diabetes?,"Based on the provided clinical notes, there is...",[Patient recorded as having No Known Allergies...,[No.]
1,What allergies did the patient have?,The patient had No Known Allergies to Drugs.,[Patient recorded as having No Known Allergies...,[The patient has no known allergies to drugs.]
2,Does the patient use tobacco or alcohol?,The patient has occasional tobacco use (cigare...,[Patient recorded as having No Known Allergies...,"[Yes, the patient occasionally uses tobacco (c..."
3,What medication is the patient taking?,The patient is taking Dilantin 100 IV TID.,[Patient recorded as having No Known Allergies...,[Dilantin 100 IV TID.]
4,What measures were taken for the patient?,"The patient, a 51-year-old with a closed head ...",[Patient recorded as having No Known Allergies...,"[The patient was intubated, had a left radial ..."


In [31]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)
     

In [32]:


def evaluate_ragas_dataset(dataset):
  result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
    
  )
  return result

In [33]:
qa_result = evaluate_ragas_dataset(dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:18<00:00, 18.10s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [00:06<00:00,  6.27s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:19<00:00, 19.72s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [34]:
print(qa_result)

{'context_precision': 0.3333, 'faithfulness': 0.8275, 'answer_relevancy': 0.9162, 'context_recall': 1.0000, 'context_relevancy': 0.1094, 'answer_correctness': 0.8624, 'answer_similarity': 1.0000}


In [35]:
df = qa_result.to_pandas()

In [36]:
df.to_csv("gpt3.5_ragas_result.csv")